In [5]:
# Install gdown to handle Google Drive downloads
!pip install gdown

# Download the specific file using its ID
!gdown --id 1lhAaeQCmk2y440PmagA0KmIVBIysVMwu -O tennis_court_det_dataset.zip



[notice] A new release of pip is available: 25.0.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\Users\rhyth\AppData\Local\Programs\Python\Python313\Lib\site-packages\gdown\__main__.py:139: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu
From (redirected): https://drive.google.com/uc?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&confirm=t&uuid=6be4e604-a797-4c2c-ac30-2ef9d9c82fc2
To: c:\Tennis Analysis\training\tennis_court_det_dataset.zip

  0%|          | 0.00/7.26G [00:00<?, ?B/s]
  0%|          | 1.57M/7.26G [00:00<08:17, 14.6MB/s]
  0%|          | 4.19M/7.26G [00:00<06:23, 18.9MB/s]
  0%|          | 6.29M/7.26G [00:00<06:42, 18.0MB/s]
  0%|          | 8.91M/7.26G [00:03<59:05, 2.04MB/s]
  0%|          | 11.0M/7.26G [00:03<41:34, 2.90MB/s]
  0%|          | 

In [8]:
import zipfile
with zipfile.ZipFile("tennis_court_det_dataset.zip", 'r') as zip_ref:
    zip_ref.extractall(".")
print("Unzipped dataset successfully.")


Unzipped dataset successfully.


In [9]:
#Start Code

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
#Create Torch Dataset

class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [12]:

train_dataset = KeypointsDataset("data/images","data/data_train.json")
val_dataset = KeypointsDataset("data/images","data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)


In [13]:
#Create Model 

model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

c:\Users\rhyth\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\rhyth\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\rhyth/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:03<00:00, 28.0MB/s]


In [14]:
model = model.to(device)


In [15]:
#Train Model

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) 

In [16]:
epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14990.8955078125
Epoch 0, iter 10, loss: 14572.4453125
Epoch 0, iter 20, loss: 15255.080078125
Epoch 0, iter 30, loss: 14143.1748046875
Epoch 0, iter 40, loss: 13029.6533203125
Epoch 0, iter 50, loss: 13184.197265625
Epoch 0, iter 60, loss: 12821.9990234375
Epoch 0, iter 70, loss: 11974.0205078125
Epoch 0, iter 80, loss: 12433.5654296875
Epoch 0, iter 90, loss: 11548.7431640625
Epoch 0, iter 100, loss: 12264.654296875
Epoch 0, iter 110, loss: 9935.94921875
Epoch 0, iter 120, loss: 10215.3076171875
Epoch 0, iter 130, loss: 10241.0810546875
Epoch 0, iter 140, loss: 9231.3232421875
Epoch 0, iter 150, loss: 9122.87109375
Epoch 0, iter 160, loss: 9115.189453125
Epoch 0, iter 170, loss: 8662.1396484375
Epoch 0, iter 180, loss: 8340.3701171875
Epoch 0, iter 190, loss: 8304.8017578125
Epoch 0, iter 200, loss: 7746.1787109375
Epoch 0, iter 210, loss: 7420.49609375
Epoch 0, iter 220, loss: 7278.58447265625
Epoch 0, iter 230, loss: 6994.6884765625
Epoch 0, iter 240, loss: 6